#Predicting sentiment from product reviews

#Fire up GraphLab Create

In [1]:
import graphlab

#Read some product review data

Loading reviews for a set of baby products. 

In [2]:
products = graphlab.SFrame('amazon_baby.gl/')

2016-05-15 17:09:01,888 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.9 started. Logging: C:\Users\Alfonso\AppData\Local\Temp\graphlab_server_1463324940.log.0


This non-commercial license of GraphLab Create is assigned to alfalive@gmail.com and will expire on May 01, 2017. For commercial licensing options, visit https://dato.com/buy/.


#Let's explore this data together

Data includes the product name, the review text and the rating of the review. 

In [3]:
products.head()

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


#Build the word count vector for each review

In [4]:
products['word_count'] = graphlab.text_analytics.count_words(products['review'])

In [5]:
products.head()

name,review,rating,word_count
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0,"{'and': 5L, 'stink': 1L,'because': 1L, 'order ..."
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3L, 'love': 1L,'it': 2L, 'highly': 1L, ..."
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2L, 'quilt': 1L,'it': 1L, 'comfortable': ..."
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'ingenious': 1L, 'and':3L, 'love': 2L, ..."
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2L, 'parents!!':1L, 'all': 2L, 'puppe ..."
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2L, 'cute': 1L,'help': 2L, 'doll': 1L, ..."
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1L, 'be': 1L,'is': 1L, 'it': 1L, ' ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'feeding,': 1L, 'and':2L, 'all': 1L, 'right': ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1L, 'help': 1L,'give': 1L, 'is': 1L, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'journal.': 1L, 'all':1L, 'standarad': 1L, ..."


In [6]:
graphlab.canvas.set_target('ipynb')

In [7]:
products['name'].show()

#Examining the reviews for most-sold product:  'Vulli Sophie the Giraffe Teether'

In [8]:
giraffe_reviews = products[products['name'] == 'Vulli Sophie the Giraffe Teether']

In [9]:
len(giraffe_reviews)

785

In [10]:
giraffe_reviews['rating'].show(view='Categorical')

#Build a sentiment classifier

In [11]:
products['rating'].show(view='Categorical')

##Define what's a positive and a negative sentiment

We will ignore all reviews with rating = 3, since they tend to have a neutral sentiment.  Reviews with a rating of 4 or higher will be considered positive, while the ones with rating of 2 or lower will have a negative sentiment.   

In [12]:
#ignore all 3* reviews
products = products[products['rating'] != 3]

In [13]:
#positive sentiment = 4* or 5* reviews
products['sentiment'] = products['rating'] >=4

In [26]:
products.head(20)

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3L, 'love': 1L,'it': 2L, 'highly': 1L, ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2L, 'quilt': 1L,'it': 1L, 'comfortable': ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'ingenious': 1L, 'and':3L, 'love': 2L, ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2L, 'parents!!':1L, 'all': 2L, 'puppe ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2L, 'cute': 1L,'help': 2L, 'doll': 1L, ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1L, 'be': 1L,'is': 1L, 'it': 1L, ' ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'feeding,': 1L, 'and':2L, 'all': 1L, 'right': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1L, 'help': 1L,'give': 1L, 'is': 1L, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'journal.': 1L, 'all':1L, 'standarad': 1L, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'all': 1L, 'forget': 1L,'just': 1L, ""daughter ...",1


##Let's train the sentiment classifier

In [27]:
train_data,test_data = products.random_split(.8, seed=0)

In [28]:
sentiment_model = graphlab.logistic_classifier.create(train_data,
                                                     target='sentiment',
                                                     features=['word_count'],
                                                     validation_set=test_data)

WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

...

| 6         | 13       | 3.000000  | 3.995568     | 0.899991          | 0.825967            |

| 10        | 18       | 1.000000  | 5.422290     | 0.988715          | 0.916256            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

#Evaluate the sentiment model

In [29]:
sentiment_model.evaluate(test_data, metric='roc_curve')

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+----------------+----------------+-------+------+
 | threshold |      fpr       |      tpr       |   p   |  n   |
 +-----------+----------------+----------------+-------+------+
 |    0.0    |      1.0       |      1.0       | 27976 | 5328 |
 |   1e-05   | 0.909346846847 | 0.998856162425 | 27976 | 5328 |
 |   2e-05   | 0.896021021021 | 0.998748927652 | 27976 | 5328 |
 |   3e-05   | 0.886448948949 | 0.998462968259 | 27976 | 5328 |
 |   4e-05   | 0.879692192192 | 0.998284243637 | 27976 | 5328 |
 |   5e-05   | 0.875187687688 | 0.998212753789 | 27976 | 5328 |
 |   6e-05   | 0.872184684685 | 0.998177008865 | 27976 | 5328 |
 |   7e-05   | 0.868618618619 | 0.998034029168 | 27976 | 5328 |
 |   8e-05   | 0.864677177177 | 0.997998284244 | 27976 | 5328 |
 |   9e-05   | 0.860735735736 | 0.997962539319 | 27976 | 5328 |
 +-----------+----------------+----------------+-------+------

In [30]:
sentiment_model.show(view='Evaluation')

### Majoritiy Class!

In [118]:
(26521+1327)/((1.0)*(26521+1327+1455+4001))

0.8361758347345665

#Applying the learned model to understand sentiment for Giraffe

In [31]:
giraffe_reviews['predicted_sentiment'] = sentiment_model.predict(giraffe_reviews, output_type='probability')

In [32]:
giraffe_reviews.head()

name,review,rating,word_count,predicted_sentiment
Vulli Sophie the GiraffeTeether ...,He likes chewing on allthe parts especially the ...,5.0,"{'and': 1L, 'all': 1L,'because': 1L, 'it': 1L, ...",0.999513023521
Vulli Sophie the GiraffeTeether ...,My son loves this toy andfits great in the diaper ...,5.0,"{'and': 1L, 'right': 1L,'help': 1L, 'just': 1L, ...",0.999320678306
Vulli Sophie the GiraffeTeether ...,There really should be alarge warning on the ...,1.0,"{'and': 2L, 'all': 1L,'latex.': 1L, 'being': ...",0.013558811687
Vulli Sophie the GiraffeTeether ...,All the moms in my moms'group got Sophie for ...,5.0,"{'and': 2L, 'one!': 1L,'all': 1L, 'love': 1L, ...",0.995769474148
Vulli Sophie the GiraffeTeether ...,I was a little skepticalon whether Sophie was ...,5.0,"{'and': 3L, 'all': 1L,'old': 1L, 'her.': 1L, ...",0.662374415673
Vulli Sophie the GiraffeTeether ...,I have been reading aboutSophie and was going ...,5.0,"{'and': 6L, 'seven': 1L,'already': 1L, 'love': ...",0.999997148186
Vulli Sophie the GiraffeTeether ...,My neice loves her sophieand has spent hours ...,5.0,"{'and': 4L, 'drooling,':1L, 'love': 1L, 'her.': ...",0.989190989536
Vulli Sophie the GiraffeTeether ...,What a friendly face!And those mesmerizing ...,5.0,"{'and': 3L, 'chew': 1L,""don't"": 1L, 'is': 1L, ...",0.999563518413
Vulli Sophie the GiraffeTeether ...,We got this just for myson to chew on instea ...,5.0,"{'chew': 2L, 'because':1L, 'just': 2L, 'what': ...",0.970160542725
Vulli Sophie the GiraffeTeether ...,"My baby seems to likethis toy, but I could ...",3.0,"{'and': 2L, 'already':1L, 'in': 1L, 'some': ...",0.195367644588


##Sort the reviews based on the predicted sentiment and explore

In [33]:
giraffe_reviews = giraffe_reviews.sort('predicted_sentiment', ascending=False)

In [34]:
giraffe_reviews.head()

name,review,rating,word_count,predicted_sentiment
Vulli Sophie the GiraffeTeether ...,"Sophie, oh Sophie, yourtime has come. My ...",5.0,"{'giggles': 1L, 'all':1L, ""violet's"": 2L, ...",1.0
Vulli Sophie the GiraffeTeether ...,I'm not sure why Sophieis such a hit with the ...,4.0,"{'peace': 1L, 'month':1L, 'bright': 1L, ...",0.999999999703
Vulli Sophie the GiraffeTeether ...,I'll be honest...I boughtthis toy because all the ...,4.0,"{'all': 2L, 'pops': 1L,'existence.': 1L, ...",0.999999999392
Vulli Sophie the GiraffeTeether ...,We got this littlegiraffe as a gift from a ...,5.0,"{'all': 2L, ""don't"": 1L,'(literally).so': 1L, ...",0.99999999919
Vulli Sophie the GiraffeTeether ...,As a mother of 16monthold twins; I bought ...,5.0,"{'cute': 1L, 'all': 1L,'reviews.': 2L, 'just': ...",0.999999998657
Vulli Sophie the GiraffeTeether ...,Sophie the Giraffe is theperfect teething toy. ...,5.0,"{'just': 2L, 'both': 1L,'month': 1L, 'ears,': ...",0.999999997108
Vulli Sophie the GiraffeTeether ...,Sophie la giraffe isabsolutely the best toy ...,5.0,"{'and': 5L, 'the': 1L,'all': 1L, 'old': 1L, ...",0.999999995589
Vulli Sophie the GiraffeTeether ...,My 5-mos old son took tothis immediately. The ...,5.0,"{'just': 1L, 'shape': 2L,'mutt': 1L, '""dog': 1L, ...",0.999999995573
Vulli Sophie the GiraffeTeether ...,My nephews and my fourkids all had Sophie in ...,5.0,"{'and': 4L, 'chew': 1L,'all': 1L, 'perfect;': ...",0.999999989527
Vulli Sophie the GiraffeTeether ...,Never thought I'd see myson French kissing a ...,5.0,"{'giggles': 1L, 'all':1L, 'out,': 1L, 'over': ...",0.999999985069


##Most positive reviews for the giraffe

In [35]:
giraffe_reviews[0]['review']

"Sophie, oh Sophie, your time has come. My granddaughter, Violet is 5 months old and starting to teeth. What joy little Sophie brings to Violet. Sophie is made of a very pliable rubber that is sturdy but not tough. It is quite easy for Violet to twist Sophie into unheard of positions to get Sophie into her mouth. The little nose and hooves fit perfectly into small mouths, and the drooling has purpose. The paint on Sophie is food quality.Sophie was born in 1961 in France. The maker had wondered why there was nothing available for babies and made Sophie from the finest rubber, phthalate-free on St Sophie's Day, thus the name was born. Since that time millions of Sophie's populate the world. She is soft and for babies little hands easy to grasp. Violet especially loves the bumpy head and horns of Sophie. Sophie has a long neck that easy to grasp and twist. She has lovely, sizable spots that attract Violet's attention. Sophie has happy little squeaks that bring squeals of delight from Viol

In [36]:
giraffe_reviews[1]['review']

"I'm not sure why Sophie is such a hit with the little ones, but my 7 month old baby girl is one of her adoring fans.  The rubber is softer and more pleasant to handle, and my daughter has enjoyed chewing on her legs and the nubs on her head even before she started teething.  She also loves the squeak that Sophie makes when you squeeze her.  Not sure what it is but if Sophie is amongst a pile of her other toys, my daughter will more often than not reach for Sophie.  And I have the peace of mind of knowing that only edible and safe paints and materials have been used to make Sophie, as opposed to Bright Starts and other baby toys made in China.  Now that the research is out on phthalates and other toxic substances in baby toys, I think it's more important than ever to find good quality toys that are also safe for our babies to handle and put in their mouths.  Sophie is a must-have for every new mom in my opinion.  Even if your kid is one of the few that can take or leave her, it's worth

##Show most negative reviews for giraffe

In [37]:
giraffe_reviews[-1]['review']

"My son (now 2.5) LOVED his Sophie, and I bought one for every baby shower I've gone to. Now, my daughter (6 months) just today nearly choked on it and I will never give it to her again. Had I not been within hearing range it could have been fatal. The strange sound she was making caught my attention and when I went to her and found the front curved leg shoved well down her throat and her face a purply/blue I panicked. I pulled it out and she vomited all over the carpet before screaming her head off. I can't believe how my opinion of this toy has changed from a must-have to a must-not-use. Please don't disregard any of the choking hazard comments, they are not over exaggerated!"

In [38]:
giraffe_reviews[-2]['review']

"This children's toy is nostalgic and very cute. However, there is a distinct rubber smell and a very odd taste, yes I tried it, that my baby did not enjoy. Also, if it is soiled it is extremely difficult to clean as the rubber is a kind of porus material and does not clean well. The final thing is the squeaking device inside which stopped working after the first couple of days. I returned this item feeling I had overpaid for a toy that was defective and did not meet my expectations. Please do not be swayed by the cute packaging and hype surounding it as I was. One more thing, I was given a full refund from Amazon without any problem."

# Quiz

## Q1.Use .apply() to build a new feature with the counts for each of the selected_words

In [39]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

In [52]:
word=''
def awesome_count(dict):
    if word in dict:
        return dict[word]
    else:
        return 0L

In [54]:
for w in selected_words:
    word = w
    products[w] = products['word_count'].apply(awesome_count)

In [58]:
products[products['hate'] > 1 ].head()

name,review,rating,word_count,sentiment,awesome
"Kiddopotamus TinydinerPlacemat, Yellow ...","I have a 24 month olddaughter, and 11 month ...",5.0,"{'all': 3L, 'managed':1L, 'silverware': 1L, ...",1,0
Cosco Alpha Omega EliteConvertible Car Seat ...,I bought this car seatafter both seeing the ...,1.0,"{'rating': 1L, 'all': 1L,'stores': 1L, 'help': ...",0,0
Playtex Embrace BreastPump System ...,If I could give 0 starsor even negative star ...,1.0,"{'would.': 1L, 'all': 1L,'just': 1L, 'money': 1L, ...",0,0
Fisher-Price PapasanCradle Swing - Nature's ...,My son loves this swing(he is 10wks) and its ...,4.0,"{'all': 1L, 'just': 1L,'over': 1L, 'while.': ...",1,0
"Stork Craft Beatrice 5Drawer Chest, White ...",I hate this dresser. Itis EXTREMELY poorly m ...,1.0,"{'and': 2L, 'own': 1L,'snugly': 1L, 'tired': ...",0,0
Discovery High Chair -Tattersall ...,I absolutely hate thisproduct. Why oh why did ...,1.0,"{'just': 3L, ""don't"": 3L,'-': 18L, 'awful': 1L, ...",0,0
"Fisher-Price Infant-To-Toddler Rocker, ...",Bought for us by myfather in law. and i ...,1.0,"{'and': 4L, 'over.': 1L,'barely': 1L, 'just': ...",0,0
Travel System - Malawi,I bought this system (ina different pattern) ...,2.0,"{'whoever': 1L, 'over':1L, 'looks': 1L, 'hate': ...",0,0
Safety 1st Easy SaverDiaper Pail ...,I hate this diaper pail.Like really hate it. ...,1.0,"{'and': 1L, 'sticking':1L, 'lid.': 1L, ...",0,0
"BABYBJORN Baby CarrierOriginal, Black, Cotton ...","I own the original BabyBjorn (classic), the ...",2.0,"{'all': 1L, 'better.':1L, 'slung': 2L, 'go': ...",0,0


In [63]:
for w in selected_words:
    print w + ':' + str(products[w].sum())

awesome:2002
great:42420
fantastic:873
amazing:1305
love:40277
horrible:659
bad:3197
terrible:673
awful:345
wow:131
hate:1057


## Q2.Create a new sentiment analysis model using only the selected_words as features:

In [66]:
train_data,test_data = products.random_split(.8, seed=0)

In [68]:
selected_words_model = graphlab.logistic_classifier.create(train_data,
                                                     target='sentiment',
                                                     features=selected_words,
                                                     validation_set=test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 11

...

| 4         | 5        | 0.322309     | 0.844269          | 0.843142            |

| 5         | 6        | 0.388821     | 0.844269          | 0.843142            |

| 6         | 7        | 0.455886     | 0.844269          | 0.843142            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

In [69]:
selected_words_model['coefficients']

name,index,class,value,stderr
(intercept),None,1,1.36728315229,0.00861805467824
awesome,None,1,1.05800888878,0.110865296265
great,None,1,0.883937894898,0.0217379527921
fantastic,None,1,0.891303090304,0.154532343591
amazing,None,1,0.892802422508,0.127989503231
love,None,1,1.39989834302,0.0287147460124
horrible,None,1,-1.99651800559,0.0973584169028
bad,None,1,-0.985827369929,0.0433603009142
terrible,None,1,-2.09049998487,0.0967241912229
awful,None,1,-1.76469955631,0.134679803365


In [75]:
selected_words_model['coefficients'].sort('value', ascending=False).print_rows(num_rows=12, num_columns=5) 

+-------------+-------+-------+------------------+------------------+
|     name    | index | class |      value       |      stderr      |
+-------------+-------+-------+------------------+------------------+
|     love    |  None |   1   |  1.39989834302   | 0.0287147460124  |
| (intercept) |  None |   1   |  1.36728315229   | 0.00861805467824 |
|   awesome   |  None |   1   |  1.05800888878   |  0.110865296265  |
|   amazing   |  None |   1   |  0.892802422508  |  0.127989503231  |
|  fantastic  |  None |   1   |  0.891303090304  |  0.154532343591  |
|    great    |  None |   1   |  0.883937894898  | 0.0217379527921  |
|     wow     |  None |   1   | -0.0541450123333 |  0.275616449416  |
|     bad     |  None |   1   | -0.985827369929  | 0.0433603009142  |
|     hate    |  None |   1   |  -1.40916406276  | 0.0771983993506  |
|    awful    |  None |   1   |  -1.76469955631  |  0.134679803365  |
|   horrible  |  None |   1   |  -1.99651800559  | 0.0973584169028  |
|   terrible  |  Non

# 3. Comparing the accuracy of different sentiment analysis model

In [76]:
selected_words_model.evaluate(test_data, metric='roc_curve')

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+-----+-----+-------+------+
 | threshold | fpr | tpr |   p   |  n   |
 +-----------+-----+-----+-------+------+
 |    0.0    | 1.0 | 1.0 | 27976 | 5328 |
 |   1e-05   | 1.0 | 1.0 | 27976 | 5328 |
 |   2e-05   | 1.0 | 1.0 | 27976 | 5328 |
 |   3e-05   | 1.0 | 1.0 | 27976 | 5328 |
 |   4e-05   | 1.0 | 1.0 | 27976 | 5328 |
 |   5e-05   | 1.0 | 1.0 | 27976 | 5328 |
 |   6e-05   | 1.0 | 1.0 | 27976 | 5328 |
 |   7e-05   | 1.0 | 1.0 | 27976 | 5328 |
 |   8e-05   | 1.0 | 1.0 | 27976 | 5328 |
 |   9e-05   | 1.0 | 1.0 | 27976 | 5328 |
 +-----------+-----+-----+-------+------+
 [100001 rows x 5 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.}

In [77]:
selected_words_model.show(view='Evaluation')

# 4. Interpreting the difference in performance between the models

In [78]:
diaper_champ_reviews = products[products['name'] == 'Baby Trend Diaper Champ']

In [79]:
len(diaper_champ_reviews)

298

In [80]:
diaper_champ_reviews['rating'].show(view='Categorical')

In [101]:
diaper_champ_reviews['predicted_sentiment'] = sentiment_model.predict(diaper_champ_reviews, output_type='probability')

In [102]:
diaper_champ_reviews = diaper_champ_reviews.sort('predicted_sentiment', ascending=False)

In [103]:
diaper_champ_reviews.head()

name,review,rating,word_count,sentiment,awesome
Baby Trend Diaper Champ,Baby Luke can turn aclean diaper to a dirty ...,5.0,"{'all': 1L, 'less': 1L,""friend's"": 1L, '(whi ...",1,0
Baby Trend Diaper Champ,I LOOOVE this diaperpail! Its the easies ...,5.0,"{'just': 1L, 'over': 1L,'rweek': 1L, 'sooo': 1L, ...",1,0
Baby Trend Diaper Champ,We researched all of thedifferent types of di ...,4.0,"{'all': 2L, 'just': 4L,""don't"": 2L, 'one,': 1L, ...",1,0
Baby Trend Diaper Champ,My baby is now 8 monthsand the can has been ...,5.0,"{""don't"": 1L, 'when': 1L,'over': 1L, 'soon': 1L, ...",1,0
Baby Trend Diaper Champ,"This is absolutely, byfar, the best diaper ...",5.0,"{'just': 3L, 'money': 1L,'not': 2L, 'mechanism': ...",1,0
Baby Trend Diaper Champ,Diaper Champ or DiaperGenie? That was my ...,5.0,"{'all': 1L, 'bags.': 1L,'son,': 1L, '(i': 1L, ...",1,0
Baby Trend Diaper Champ,Wow! This is fabulous.It was a toss-up between ...,5.0,"{'and': 4L, '""genie"".':1L, 'since': 1L, ...",1,0
Baby Trend Diaper Champ,I originally put thisitem on my baby registry ...,5.0,"{'lysol': 1L, 'all': 2L,'bags.': 1L, 'feedback': ...",1,0
Baby Trend Diaper Champ,Two girlfriends and twofamily members put me ...,5.0,"{'just': 1L, 'when': 1L,'both': 1L, 'results': ...",1,0
Baby Trend Diaper Champ,I am one of those super-critical shoppers who ...,5.0,"{'taller': 1L, 'bags.':1L, 'just': 1L, ""don't"": ...",1,0


In [104]:
diaper_champ_reviews[0]['review']

'Baby Luke can turn a clean diaper to a dirty diaper in 3 seconds flat. The diaper champ turns the smelly diaper into "what diaper smell" in less time than that. I hesitated and wondered what I REALLY needed for the nursery. This is one of the best purchases we made. The champ, the baby bjorn, fluerville diaper bag, and graco pack and play bassinet all vie for the best baby purchase.Great product, easy to use, economical, effective, absolutly fabulous.UpdateI knew that I loved the champ, and useing the diaper genie at a friend\'s house REALLY reinforced that!! There is no comparison, the chanp is easy and smell free, the genie was difficult to use one handed (which is absolutly vital if you have a little one on a changing pad) and there was a deffinite odor eminating from the genieplus we found that the quick tie garbage bags where the ties are integrated into the bag work really well because there isn\'t any added bulk around the sealing edge of the champ.'

In [105]:
diaper_champ_reviews[0]['word_count']

{'"what': 1L,
 '(which': 1L,
 '3': 1L,
 'a': 6L,
 'absolutly': 2L,
 'added': 1L,
 'all': 1L,
 'and': 6L,
 'any': 1L,
 'are': 1L,
 'around': 1L,
 'at': 1L,
 'baby': 3L,
 'bag': 1L,
 'bag,': 1L,
 'bags': 1L,
 'bassinet': 1L,
 'because': 1L,
 'best': 2L,
 'bjorn,': 1L,
 'bulk': 1L,
 'can': 1L,
 'champ': 1L,
 'champ,': 2L,
 'champ.': 1L,
 'changing': 1L,
 'chanp': 1L,
 'clean': 1L,
 'comparison,': 1L,
 'deffinite': 1L,
 'diaper': 7L,
 'difficult': 1L,
 'dirty': 1L,
 'easy': 2L,
 'economical,': 1L,
 'edge': 1L,
 'effective,': 1L,
 'eminating': 1L,
 'fabulous.updatei': 1L,
 'flat.': 1L,
 'fluerville': 1L,
 'for': 2L,
 'found': 1L,
 'free,': 1L,
 "friend's": 1L,
 'from': 1L,
 'garbage': 1L,
 'genie': 2L,
 'genieplus': 1L,
 'graco': 1L,
 'handed': 1L,
 'have': 1L,
 'hesitated': 1L,
 'house': 1L,
 'i': 3L,
 'if': 1L,
 'in': 2L,
 'integrated': 1L,
 'into': 2L,
 'is': 4L,
 "isn't": 1L,
 'knew': 1L,
 'less': 1L,
 'little': 1L,
 'loved': 1L,
 'luke': 1L,
 'made.': 1L,
 'needed': 1L,
 'no': 1L,
 'nu

In [106]:
for w in selected_words:
    print w + ':' + str(diaper_champ_reviews[0][w])

awesome:0
great:0
fantastic:0
amazing:0
love:0
horrible:0
bad:0
terrible:0
awful:0
wow:0
hate:0


In [110]:
diaper_champ_reviews['predicted_sentiment_simpler'] = selected_words_model.predict(diaper_champ_reviews, output_type='probability')

In [111]:
diaper_champ_reviews = diaper_champ_reviews.sort('predicted_sentiment_simpler', ascending=False)

In [112]:
diaper_champ_reviews.head()

name,review,rating,word_count,sentiment,awesome
Baby Trend Diaper Champ,I LOVE LOVE LOVE thisproduct! It is SO much ...,4.0,"{'rating': 1L,'contacted': 1L, 'over': ...",1,0
Baby Trend Diaper Champ,"Love it, love it, loveit! This lives up to ...",5.0,"{'all': 1L, 'already':1L, 'love': 3L, 'have': ...",1,0
Baby Trend Diaper Champ,I received my DiaperChamp at my baby shower ...,5.0,"{'bags.': 1L, ""don't"":1L, 'son.': 1L, 'of,': ...",1,0
Baby Trend Diaper Champ,I love this diaper paleand wouldn't dream of ...,5.0,"{'and': 3L, 'love': 1L,'use.': 1L, 'is': 2L, ...",1,0
Baby Trend Diaper Champ,Works great - no smells.LOVE that it uses reg ...,5.0,"{'and': 2L, 'love': 1L,'garbage': 1L, ...",1,0
Baby Trend Diaper Champ,"This is absolutely, byfar, the best diaper ...",5.0,"{'just': 3L, 'money': 1L,'not': 2L, 'mechanism': ...",1,0
Baby Trend Diaper Champ,We had 2 diaper Genie'sboth given to us as a ...,4.0,"{'hand.': 1L,'(required': 1L, ...",1,0
Baby Trend Diaper Champ,I love this diaper pail!It's so easy to use a ...,5.0,"{'and': 3L, 'this': 1L,'love': 2L, 'being': 1L, ...",1,0
Baby Trend Diaper Champ,I have a 10 year olddaughter and an 8 month ...,5.0,"{'lysol': 1L, 'all': 1L,'just': 2L, 'hand,': 1L, ...",1,0
Baby Trend Diaper Champ,I love this diaper pail.It keeps the diapers ...,4.0,"{'and': 1L, 'bags.': 1L,'extra': 1L, 'is': 1L, ...",1,0


In [113]:
diaper_champ_reviews[0]['review']

'I LOVE LOVE LOVE this product! It is SO much easier to use than the Diaper Genie, (you need a PHD in poopy to figure out how to use the darn thing!) and it even takes the same bags as my kitchen trash can, shich is super convenient, and cost efficient as I can buy them in bulk.The only reason for not rating it a 5 star was that I did have one small problem with it. The foam gasket in the barrell which keeps the poopy smell inside the unit ripped somehow, and it got VERY stinky. HOWEVER, I contacted the manufacturer though their website, and received an email back the same day stating that this was unusual, and that replacement gaskets were on their way to me. They arrived inside of a week and after replacing, it works great again! (They even sent me extras should it happen again)I HIGHLY reccomend this diaper pail over ANY competitors, you will not be sorry!'

In [114]:
diaper_champ_reviews[0]['word_count']

{'(they': 1L,
 '(you': 1L,
 '5': 1L,
 'a': 3L,
 'after': 1L,
 'again!': 1L,
 'again)i': 1L,
 'an': 1L,
 'and': 6L,
 'any': 1L,
 'arrived': 1L,
 'as': 2L,
 'back': 1L,
 'bags': 1L,
 'barrell': 1L,
 'be': 1L,
 'bulk.the': 1L,
 'buy': 1L,
 'can': 1L,
 'can,': 1L,
 'competitors,': 1L,
 'contacted': 1L,
 'convenient,': 1L,
 'cost': 1L,
 'darn': 1L,
 'day': 1L,
 'diaper': 2L,
 'did': 1L,
 'easier': 1L,
 'efficient': 1L,
 'email': 1L,
 'even': 2L,
 'extras': 1L,
 'figure': 1L,
 'foam': 1L,
 'for': 1L,
 'gasket': 1L,
 'gaskets': 1L,
 'genie,': 1L,
 'got': 1L,
 'great': 1L,
 'happen': 1L,
 'have': 1L,
 'highly': 1L,
 'how': 1L,
 'however,': 1L,
 'i': 4L,
 'in': 3L,
 'inside': 2L,
 'is': 2L,
 'it': 6L,
 'it.': 1L,
 'keeps': 1L,
 'kitchen': 1L,
 'love': 3L,
 'manufacturer': 1L,
 'me': 1L,
 'me.': 1L,
 'much': 1L,
 'my': 1L,
 'need': 1L,
 'not': 2L,
 'of': 1L,
 'on': 1L,
 'one': 1L,
 'only': 1L,
 'out': 1L,
 'over': 1L,
 'pail': 1L,
 'phd': 1L,
 'poopy': 2L,
 'problem': 1L,
 'product!': 1L,
 'rati

In [115]:
for w in selected_words:
    print w + ':' + str(diaper_champ_reviews[0][w])

awesome:0
great:1
fantastic:0
amazing:0
love:3
horrible:0
bad:0
terrible:0
awful:0
wow:0
hate:0
